In [31]:
import torch 
import torch.nn.functional as F
import torch.nn as nn

from torch.utils.data import Dataset, DataLoader

import numpy as np
from numpy.random import normal

from MNISTDataset import MNISTDataset
from sklearn import datasets

torch.manual_seed(17)
np.random.seed(17)

N = 1600
batch = 32

digits = datasets.load_digits()

g_train = MNISTDataset(digits.data[:1600], digits.target[:1600], N)
data_loader = DataLoader(g_train, batch_size =  32, shuffle=True)

g_test = MNISTDataset(digits.data[1600:], digits.target[1600:], 1765-N)
test_loader = DataLoader(g_test, batch_size =  32, shuffle=True)

# simple NN:
class NN(nn.Module):   
    def __init__(self, n1, n2, n3):
        super().__init__()
        self.lin1 = nn.Linear(n1, n2)
        self.act = nn.ReLU()        
        self.lin3 = nn.Linear(n2, n3)
                
    def forward(self, x):
        out = self.lin1(x)
        out = self.act(out)
        out = self.lin3(out)

        return out 
    
inp = 8

net = NN(inp*inp, 100, 10)

In [ ]:
for n, p in net.named_parameters():
    print("")

In [29]:


loss = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.001)

for e in range(10):
    it_ = iter(data_loader)
    for i in range(50) :
        x, y = next(it_)
        
        x = x.float().reshape(-1, inp*inp).requires_grad_()
        y = torch.argmax(y, dim=1)
        
        out = net(x)
        
        l = loss(out, y)

        print(f"Epoch {e+1} Iteration {i} Loss {l}")

        optimizer.zero_grad()
        l.backward()
        optimizer.step()

Epoch 1 Iteration 0 Loss 3.926954507827759
Epoch 1 Iteration 1 Loss 3.306882619857788
Epoch 1 Iteration 2 Loss 3.52573823928833
Epoch 1 Iteration 3 Loss 3.4346632957458496
Epoch 1 Iteration 4 Loss 3.4031739234924316
Epoch 1 Iteration 5 Loss 3.3136603832244873
Epoch 1 Iteration 6 Loss 3.098027229309082
Epoch 1 Iteration 7 Loss 3.343773603439331
Epoch 1 Iteration 8 Loss 3.1698598861694336
Epoch 1 Iteration 9 Loss 3.3562471866607666
Epoch 1 Iteration 10 Loss 2.755659818649292
Epoch 1 Iteration 11 Loss 2.8606362342834473
Epoch 1 Iteration 12 Loss 2.7058730125427246
Epoch 1 Iteration 13 Loss 2.437093496322632
Epoch 1 Iteration 14 Loss 3.0814177989959717
Epoch 1 Iteration 15 Loss 2.823507308959961
Epoch 1 Iteration 16 Loss 2.674312114715576
Epoch 1 Iteration 17 Loss 2.9254772663116455
Epoch 1 Iteration 18 Loss 2.484011173248291
Epoch 1 Iteration 19 Loss 2.579915761947632
Epoch 1 Iteration 20 Loss 2.6552836894989014
Epoch 1 Iteration 21 Loss 2.7920103073120117
Epoch 1 Iteration 22 Loss 2.6165

Epoch 4 Iteration 47 Loss 1.0068241357803345
Epoch 4 Iteration 48 Loss 0.7762796878814697
Epoch 4 Iteration 49 Loss 0.8388192057609558
Epoch 5 Iteration 0 Loss 0.8751064538955688
Epoch 5 Iteration 1 Loss 0.8132912516593933
Epoch 5 Iteration 2 Loss 0.8409005403518677
Epoch 5 Iteration 3 Loss 1.0143119096755981
Epoch 5 Iteration 4 Loss 0.8430842161178589
Epoch 5 Iteration 5 Loss 0.9215313792228699
Epoch 5 Iteration 6 Loss 0.6538724303245544
Epoch 5 Iteration 7 Loss 1.0033366680145264
Epoch 5 Iteration 8 Loss 0.8117235898971558
Epoch 5 Iteration 9 Loss 0.9651161432266235
Epoch 5 Iteration 10 Loss 1.0185624361038208
Epoch 5 Iteration 11 Loss 0.8018267750740051
Epoch 5 Iteration 12 Loss 0.8676580190658569
Epoch 5 Iteration 13 Loss 0.8939915299415588
Epoch 5 Iteration 14 Loss 0.8649663925170898
Epoch 5 Iteration 15 Loss 0.7371720671653748
Epoch 5 Iteration 16 Loss 0.7453422546386719
Epoch 5 Iteration 17 Loss 0.9951702356338501
Epoch 5 Iteration 18 Loss 0.7936350107192993
Epoch 5 Iteration 19

Epoch 9 Iteration 3 Loss 0.5907174944877625
Epoch 9 Iteration 4 Loss 0.5392916202545166
Epoch 9 Iteration 5 Loss 0.5612115859985352
Epoch 9 Iteration 6 Loss 0.4423549771308899
Epoch 9 Iteration 7 Loss 0.33490878343582153
Epoch 9 Iteration 8 Loss 0.34671837091445923
Epoch 9 Iteration 9 Loss 0.4592186510562897
Epoch 9 Iteration 10 Loss 0.39029204845428467
Epoch 9 Iteration 11 Loss 0.5741882920265198
Epoch 9 Iteration 12 Loss 0.5588272213935852
Epoch 9 Iteration 13 Loss 0.588877260684967
Epoch 9 Iteration 14 Loss 0.35372617840766907
Epoch 9 Iteration 15 Loss 0.47406038641929626
Epoch 9 Iteration 16 Loss 0.41723722219467163
Epoch 9 Iteration 17 Loss 0.424917608499527
Epoch 9 Iteration 18 Loss 0.47665783762931824
Epoch 9 Iteration 19 Loss 0.5742712616920471
Epoch 9 Iteration 20 Loss 0.43611952662467957
Epoch 9 Iteration 21 Loss 0.536885678768158
Epoch 9 Iteration 22 Loss 0.3453541398048401
Epoch 9 Iteration 23 Loss 0.5312268137931824
Epoch 9 Iteration 24 Loss 0.6141644716262817
Epoch 9 Iter

In [30]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for images, labels in test_loader:
        images = images.reshape(-1, inp*inp).float()
        labels = torch.argmax(labels, dim=1)
        outputs = net(images)
        
        _, predicted = torch.max(outputs.data, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network on the 10000 test images: {acc} %')

Accuracy of the network on the 10000 test images: 85.45454545454545 %
